# Резервация на хотел с междинен софтуер за приоритетни членове

Този бележник демонстрира **функционален междинен софтуер** с използване на Microsoft Agent Framework. Разширяваме примера с условен работен процес чрез добавяне на междинен слой, който предоставя специални привилегии на приоритетните членове.

## Какво ще научите:
1. **Функционален междинен софтуер**: Прехващане и модифициране на резултати от функции
2. **Достъп до контекст**: Четене и модифициране на `context.result` след изпълнение
3. **Имплементиране на бизнес логика**: Ползи за приоритетните членове
4. **Преодоляване на резултата**: Промяна на резултатите от функцията според потребителския статус
5. **Същият работен процес, различни резултати**: Промени в поведението, водени от междинния софтуер

## Архитектура на работния процес с междинен софтуер:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Основна разлика спрямо условния работен процес:

**Без междинен софтуер** (14-conditional-workflow.ipynb):
- Париж няма налични стаи → Пренасочване към alternative_agent

**С междинен софтуер** (този бележник):
- Обикновен потребител + Париж → Няма стаи → Пренасочване към alternative_agent
- Приоритетен потребител + Париж → 🌟 Междинният софтуер преодолява! → Налично → Пренасочване към booking_agent

## Предпоставки:
- Инсталиран Microsoft Agent Framework
- Разбиране на условни работни процеси (виж 14-conditional-workflow.ipynb)
- GitHub токен или OpenAI API ключ
- Основни познания за междинни софтуерни модели


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Стъпка 1: Дефинирайте Pydantic модели за структурирани изходи

Тези модели дефинират **схемата**, която агентите ще връщат. Добавихме поле `priority_override`, за да проследяваме кога междинният софтуер променя резултата за наличност.


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Стъпка 2: Дефиниране на база данни с приоритетни членове

За това демо ще симулираме база данни с приоритетни членове. В продукция това би заявило към реална база данни или API.

**Приоритетни членове:**
- `alice@example.com` - VIP член
- `bob@example.com` - Премиум член  
- `priority_user` - Тестов акаунт


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Стъпка 3: Създайте инструмента за резервация на хотели

Същото като условния работен процес, но сега ще бъде прихванат от междинен софтуер!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Стъпка 4: 🌟 Създаване на междинен софтуер за проверка на приоритет (КЛЮЧОВАТА ФУНКЦИЯ!)

Това е **основната функционалност** на тази тетрадка. Междинният софтуер:

1. **Прекъсва** извикването на функцията hotel_booking
2. **Изпълнява** функцията нормално чрез извикване на `next(context)`
3. **Проверява** резултата в `context.result`
4. **Заменя** резултата, ако потребителят е с приоритет и няма налични стаи
5. **Връща** модифицирания резултат обратно на агента

**Ключов шаблон:**
```python
async def my_middleware(context, next):
    await next(context)  # Изпълни функция
    # Сега context.result съдържа изхода на функцията
    if some_condition:
        context.result = new_value  # Замени!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Стъпка 5: Дефиниране на функции за условия за маршрутизация

Същите функции за условия като в условния работен поток - те проверяват структурирания изход, за да определят маршрутизацията.


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Стъпка 6: Създаване на персонализиран дисплей екзекутор

Същият екзекутор като преди - показва крайния изход от работния процес.


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Стъпка 7: Зареждане на променливи на средата

Конфигурирайте LLM клиента (GitHub Models или OpenAI).


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Стъпка 8: Създаване на AI агенти с междинен софтуер

**КЛЮЧОВА РАЗЛИКА:** Когато създаваме availability_agent, ние подаваме параметъра `middleware`!

Така внедряваме priority_check_middleware в конвейера за извикване на функцията на агента.


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Стъпка 9: Изграждане на работния процес

Същата структура на работния процес както преди - условно маршрутизиране въз основа на наличността.


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Стъпка 10: Тестов случай 1 - Обикновен потребител в Париж (Без презаписване)

Обикновен потребител се опитва да резервира Париж → Няма стаи → Пренасочва към alternative_agent


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Стъпка 11: Тестови случай 2 - 🌟 Потребител с приоритет в Париж (С ПРЕЗАПИСВАНЕ!)

Член с приоритет се опитва да резервира Париж → Първоначално няма налични стаи → 🌟 Middleware прави презаписване! → Пренасочва към booking_agent

**Това е ключовото демонстриране на силата на middleware!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Стъпка 12: Тестов случай 3 - Приоритетен потребител в Стокхолм (Вече наличен)

Приоритетният потребител избира Стокхолм → Има налични стаи → Няма нужда от презаписване → Пренасочва се към booking_agent

Това показва, че междинният софтуер действа само когато е необходимо!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## Основни изводи и концепции за междинен слой

### ✅ Какво научихте:

#### **1. Шаблон за междинен слой, базиран на функции**

Междинният слой прихваща извиквания на функции, използвайки проста асинхронна функция:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Преди изпълнение на функцията
    print("Intercepting...")
    
    # Изпълнете функцията
    await next(context)
    
    # След изпълнение на функцията - прегледайте резултата
    if context.result:
        # Модифицирайте резултата, ако е необходимо
        context.result = modified_value
```

#### **2. Достъп до контекст и замяна на резултата**

- `context.function` - Достъп до функцията, която се извиква
- `context.arguments` - Четене на аргументите на функцията
- `context.kwargs` - Достъп до допълнителни параметри
- `await next(context)` - Изпълнение на функцията
- `context.result` - Четене/промяна на изхода от функцията

#### **3. Имплементация на бизнес логика**

Нашият междинен слой имплементира приоритетни привилегии за членове:
- **Обикновени потребители**: Без промени, стандартен работен поток
- **Приоритетни потребители**: Замества "няма наличност" с "налично"
- **Условна логика**: Замени само когато е необходимо

#### **4. Един и същ работен поток, различни резултати**

Силата на междинния слой:
- ✅ Без промени в структурата на работния поток
- ✅ Без промени във функцията на инструмента
- ✅ Без промени в логиката на условното маршрутизиране
- ✅ Само междинен слой → Различно поведение!

### 🚀 Приложения в реалния свят:

1. **VIP/Премиум функции**
   - Замяна на лимити за скорост за премиум потребители
   - Приоритетен достъп до ресурси
   - Динамично отключване на премиум функции

2. **A/B тестване**
   - Маршрутизиране на потребители към различни реализации
   - Тест на нови функции с конкретни потребители
   - Плавно внедряване на функции

3. **Сигурност и съответствие**
   - Одит на извиквания на функции
   - Блокиране на чувствителни операции
   - Изпълнение на бизнес правила

4. **Оптимизация на производителността**
   - Кеширане на резултати за конкретни потребители
   - Пропускане на скъпи операции, когато е възможно
   - Динамично разпределение на ресурси

5. **Обработка на грешки и повторни опити**
   - Улавяне и обработка на грешки безпроблемно
   - Имплементиране на логика за повторни опити
   - Резервен план с алтернативни реализации

6. **Логване и мониторинг**
   - Следене на време за изпълнение на функции
   - Логване на параметри и резултати
   - Мониторинг на модели на използване

### 🔑 Основни разлики спрямо декоратори:

| Функция | Декоратор | Междинен слой |
|---------|-----------|---------------|
| **Обхват** | Една функция | Всички функции в агента |
| **Гъвкавост** | Фиксиран при дефиниция | Динамичен по време на изпълнение |
| **Контекст** | Ограничен | Пълен контекст на агента |
| **Съставяне** | Множество декоратори | Конвейер на междинния слой |
| **Осъзнатост на агента** | Не | Да (достъп до състоянието на агента) |

### 📚 Кога да използвате междинен слой:

✅ **Използвайте междинен слой когато:**
- Трябва да промените поведението базирано на състоянието на потребителя/сесията
- Искате да приложите логика към множество функции
- Трябва да имате достъп до контекст на ниво агент
- Имплементирате функции с общо значение (логване, удостоверяване и др.)

❌ **Не използвайте междинен слой когато:**
- Проста валидация на входни данни (използвайте Pydantic)
- Логика специфична за функцията (задръжте в самата функция)
- Еднократни промени (променете директно функцията)

### 🎓 Разширени шаблони:

```python
# Множество междинни софтуерни слоеве (редът на изпълнение има значение!)
middleware=[
    logging_middleware,      # Първо логове
    auth_middleware,         # След това проверка на удостоверяване
    cache_middleware,        # След това проверка на кеш
    rate_limit_middleware,   # След това ограничаване на честотата
    priority_check_middleware  # Накрая проверка на приоритет
]

# Условно изпълнение на междинен софтуер
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Промяна на резултата
    else:
        # Пропускане на изпълнението изцяло
        context.result = cached_value
```

### 🔗 Свързани концепции:

- **Агенти междинен слой**: Прихваща извиквания на agent.run()
- **Междинен слой на функции**: Прихваща извиквания на функции на инструменти (какъвто използвахме!)
- **Конвейер на междинния слой**: Верига от междинни слоеве, изпълняващи се поред
- **Пропагация на контекст**: Предаване на състояние през веригата на междинния слой


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от отговорност**:
Този документ е преведен с помощта на AI преводаческа услуга [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи могат да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или неправилни тълкувания, възникнали вследствие на използването на този превод.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
